## <font color=green> Agregação de dados e operações em grupos

- O pandas oferece uma interface flexível groupby, que permite manipular e resumir conjuntos de dados de forma flexível

In [1]:
import pandas as pd
import numpy as np

### Funcionamento de GroupBy

In [2]:
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                   'key2': ['one','two','one','two','one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-1.596306,-1.417271
1,a,two,1.858539,-1.123018
2,b,one,-0.013897,2.656044
3,b,two,-0.286814,0.864779
4,a,one,-0.835811,-0.366883


- Suponha que quiséssemos calcular a média da coluna data1 usando os rótulos de key1

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

- grouped é um objeto GroupBy
- A ideia é que esse objeto tenha todas as informações necessárias para então aplicar alguma operação em cada um dos grupos

- Para calcular as médias dos grupos, podemos chamar o método mean

In [4]:
grouped.mean()

key1
a   -0.191193
b   -0.150356
Name: data1, dtype: float64

- Observe que os dados (uma Series) foram agregados de acordo com a chave de grupo, gerando uma nova Series, que agora está indexado pelos valores únicos da coluna key1

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -1.216059
      two     1.858539
b     one    -0.013897
      two    -0.286814
Name: data1, dtype: float64

- Nesse caso, agrupamos os dados usando duas chaves, e a Series resultante agora tem um índice hierárquico constituído dos pares de chave únicos observados

In [6]:
means.unstack()

key2,one,two
key1,,
a,-1.216059,1.858539
b,-0.013897,-0.286814


- É possível passar os nomes das colunas como as chaves de grupo

In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.191193,-0.969057
b,-0.150356,1.760411


In [8]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.216059 -0.892077
     two   1.858539 -1.123018
b    one  -0.013897  2.656044
     two  -0.286814  0.864779

- Perceba que no primeiro caso, não há nenhuma coluna key2. Ela foi excluída do resultado.


- Outro método útil é size, que devolve uma Series contendo os tamanhos dos grupos

In [9]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterando por grupos

- O objeto GroupBy aceita iteração, gerando uma sequência de tuplas de 2 contendo o nome do grupo, junto com a porção de dados

In [10]:
df

,key1,key2,data1,data2
0,a,one,-1.596306,-1.417271
1,a,two,1.858539,-1.123018
2,b,one,-0.013897,2.656044
3,b,two,-0.286814,0.864779
4,a,one,-0.835811,-0.366883


In [11]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.596306 -1.417271
1    a  two  1.858539 -1.123018
4    a  one -0.835811 -0.366883
b
  key1 key2     data1     data2
2    b  one -0.013897  2.656044
3    b  two -0.286814  0.864779


In [12]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.596306 -1.417271
4    a  one -0.835811 -0.366883
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.858539 -1.123018
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.013897  2.656044
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.286814  0.864779


- Outra forma de iterar por um objeto GroupBy é utilizando sua função __iter__ e __next__

In [13]:
generator = df.groupby(['key1']).__iter__()

In [14]:
group_id, grouped_data = generator.__next__()
print(group_id) 
grouped_data

a


,key1,key2,data1,data2
0,a,one,-1.596306,-1.417271
1,a,two,1.858539,-1.123018
4,a,one,-0.835811,-0.366883


In [15]:
group_id, grouped_data = generator.__next__()
print(group_id) 
grouped_data

b


,key1,key2,data1,data2
2,b,one,-0.013897,2.656044
3,b,two,-0.286814,0.864779


- Pode ser útil gerar um dicionário de porções de dados usando um só linha de código

In [16]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -1.596306 -1.417271
 1    a  two  1.858539 -1.123018
 4    a  one -0.835811 -0.366883,
 'b':   key1 key2     data1     data2
 2    b  one -0.013897  2.656044
 3    b  two -0.286814  0.864779}

In [17]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.013897,2.656044
3,b,two,-0.286814,0.864779


### Selecionando uma coluna ou um subconjunto de colunas

- Retornando uma Series

In [18]:
df.groupby(['key1', 'key2'])['data1'].mean()

key1  key2
a     one    -1.216059
      two     1.858539
b     one    -0.013897
      two    -0.286814
Name: data1, dtype: float64

- Retornando um DataFrame

In [19]:
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -1.216059
     two   1.858539
b    one  -0.013897
     two  -0.286814

### Agrupando com dicionários e Series

In [20]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a','b','c','d','e'], 
                      index=['Joe','Steve','Wes','Jim','Mike'])
people

,a,b,c,d,e
Joe,1.120521,1.057409,0.311866,0.832979,-0.420623
Steve,1.483673,-0.012796,-1.212326,0.292828,1.338678
Wes,0.945405,1.695613,-0.281598,-1.148175,0.190669
Jim,-0.439207,0.954236,-1.255734,-0.348438,-0.120391
Mike,0.816962,0.867745,0.649195,0.448024,1.754749


In [21]:
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,1.120521,1.057409,0.311866,0.832979,-0.420623
Steve,1.483673,-0.012796,-1.212326,0.292828,1.338678
Wes,0.945405,NaN,NaN,-1.148175,0.190669
Jim,-0.439207,0.954236,-1.255734,-0.348438,-0.120391
Mike,0.816962,0.867745,0.649195,0.448024,1.754749


In [22]:
mapping = {'a': 'red', 'b':'red', 'c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}

In [23]:
people.groupby(mapping,axis=1).sum()

,blue,red
Joe,1.144845,1.757308
Steve,-0.919497,2.809555
Wes,-1.148175,1.136074
Jim,-1.604172,0.394638
Mike,1.097218,3.439456


- Note que construímos um array a partir de um dicionário que foi passado no groupby

- A mesma funcionalidade vale para Series, que pode ser vista como um mapeamento de tamanho fixo

In [24]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [25]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Mike,2,3


### Agrupando com funções

- Suponha que quiséssemos agrupar pelo tamanho dos nomes das pessoas (índices do dataframe), basta usar a função len

In [26]:
people

,a,b,c,d,e
Joe,1.120521,1.057409,0.311866,0.832979,-0.420623
Steve,1.483673,-0.012796,-1.212326,0.292828,1.338678
Wes,0.945405,NaN,NaN,-1.148175,0.190669
Jim,-0.439207,0.954236,-1.255734,-0.348438,-0.120391
Mike,0.816962,0.867745,0.649195,0.448024,1.754749


In [27]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.626720,2.011645,-0.943868,-0.663634,-0.350345
4,0.816962,0.867745,0.649195,0.448024,1.754749
5,1.483673,-0.012796,-1.212326,0.292828,1.338678


### Agregação de dados

- Métodos otimizados de groupby

In [28]:
df

,key1,key2,data1,data2
0,a,one,-1.596306,-1.417271
1,a,two,1.858539,-1.123018
2,b,one,-0.013897,2.656044
3,b,two,-0.286814,0.864779
4,a,one,-0.835811,-0.366883


In [29]:
df.groupby(['key1']).count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [30]:
df.groupby(['key1']).sum()

,data1,data2
key1,,
a,-0.573578,-2.907172
b,-0.300711,3.520823


In [31]:
df.groupby(['key1']).mean()

,data1,data2
key1,,
a,-0.191193,-0.969057
b,-0.150356,1.760411


In [32]:
df.groupby(['key1']).median()

,data1,data2
key1,,
a,-0.835811,-1.123018
b,-0.150356,1.760411


In [33]:
df.groupby(['key1']).std()

,data1,data2
key1,,
a,1.815389,0.541855
b,0.192981,1.266616


In [34]:
df.groupby(['key1']).min()

,key2,data1,data2
key1,,,
a,one,-1.596306,-1.417271
b,one,-0.286814,0.864779


In [35]:
df.groupby(['key1']).max()

,key2,data1,data2
key1,,,
a,two,1.858539,-0.366883
b,two,-0.013897,2.656044


In [36]:
df.groupby(['key1']).prod()

,data1,data2
key1,,
a,2.479682,-0.583939
b,0.003986,2.296890


In [37]:
df.groupby(['key1']).first()

,key2,data1,data2
key1,,,
a,one,-1.596306,-1.417271
b,one,-0.013897,2.656044


In [38]:
df.groupby(['key1']).last()

,key2,data1,data2
key1,,,
a,one,-0.835811,-0.366883
b,two,-0.286814,0.864779


- É possível também definir as agregações a serem feitas. Por exemplo, é possível quantis de amostragem em uma Séries ou em colunas de DataFrame.

In [39]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.319669
b   -0.041189
Name: data1, dtype: float64

- Para usar suas próprias funções de agregação, passe qualquer função que agregue um array para o método aggregate ou agg

In [40]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [41]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.454845,1.050388
b,0.272917,1.791265


### Aplicação de função nas colunas e aplicação de várias funções

- Vamos voltar a usar os dados do IBGE

In [42]:
dados = pd.read_csv("datasets/dados.csv")
dados

,UF,Sexo,Idade,Cor,Anos de Estudo,Renda,Altura
0,11,0,23,8,12,800,1.603808
1,11,1,23,2,12,1150,1.739790
2,11,1,35,8,15,880,1.760444
3,11,0,46,2,6,3500,1.783158
4,11,1,47,8,9,150,1.690631
...,...,...,...,...,...,...,...
76835,53,1,46,2,11,812,1.687030
76836,53,0,30,4,7,1500,1.792934
76837,53,0,32,8,12,1300,1.830587
76838,53,0,57,8,4,1500,1.726344


- Acrescentar legenda para as colunas Sexo e Cor

In [43]:
def get_cor(x):
    cor = {0:"Indígena", 2:"Branca", 4:"Preta", 6:"Amarela", 8:"Parda", 9:"Sem declaração"}
    return cor[x]

In [44]:
dados['Gênero'] = dados['Sexo'].apply(lambda x: "Feminino" if x else "Masculino")
dados['Cor2'] = dados['Cor'].apply(get_cor)

In [45]:
dados

,UF,Sexo,Idade,Cor,Anos de Estudo,Renda,Altura,Gênero,Cor2
0,11,0,23,8,12,800,1.603808,Masculino,Parda
1,11,1,23,2,12,1150,1.739790,Feminino,Branca
2,11,1,35,8,15,880,1.760444,Feminino,Parda
3,11,0,46,2,6,3500,1.783158,Masculino,Branca
4,11,1,47,8,9,150,1.690631,Feminino,Parda
...,...,...,...,...,...,...,...,...,...
76835,53,1,46,2,11,812,1.687030,Feminino,Branca
76836,53,0,30,4,7,1500,1.792934,Masculino,Preta
76837,53,0,32,8,12,1300,1.830587,Masculino,Parda
76838,53,0,57,8,4,1500,1.726344,Masculino,Parda


- Agrupando pelas colunas que acabamos de acrescentar

In [46]:
grouped = dados.groupby(['Gênero','Cor2'])

- Observe que, para estatísticas descritivas, podemos passa o nome da função como uma string

In [47]:
grouped_renda = grouped['Renda']
grouped_renda.agg('mean')

Gênero     Cor2    
Feminino   Amarela     3027.341880
           Branca      2109.866750
           Indígena    2464.386139
           Parda       1176.758516
           Preta       1134.596400
Masculino  Amarela     4758.251064
           Branca      2925.744435
           Indígena    1081.710938
           Parda       1659.577425
           Preta       1603.861687
Name: Renda, dtype: float64

- Podemos passar uma lista de funções também

In [48]:
grouped_renda.agg(['mean','std',peak_to_peak])

mean           std  peak_to_peak
Gênero    Cor2                                             
Feminino  Amarela   3027.341880   3731.173660         20000
          Branca    2109.866750   3251.013154        100000
          Indígena  2464.386139  11957.498292        120000
          Parda     1176.758516   1596.233048         30000
          Preta     1134.596400   1349.799809         23000
Masculino Amarela   4758.251064   5740.824820         50000
          Branca    2925.744435   4750.791872        200000
          Indígena  1081.710938   1204.093490         10000
          Parda     1659.577425   2312.087184        100000
          Preta     1603.861687   1936.309271         50000

- É possível alterar o nome das funções que são exibidas no DataFrame

In [49]:
grouped_renda.agg([('Média','mean'),('Desvio Padrão',np.std)])

Média  Desvio Padrão
Gênero    Cor2                                
Feminino  Amarela   3027.341880    3731.173660
          Branca    2109.866750    3251.013154
          Indígena  2464.386139   11957.498292
          Parda     1176.758516    1596.233048
          Preta     1134.596400    1349.799809
Masculino Amarela   4758.251064    5740.824820
          Branca    2925.744435    4750.791872
          Indígena  1081.710938    1204.093490
          Parda     1659.577425    2312.087184
          Preta     1603.861687    1936.309271

- Podemos especificar uma lista de funçõs a serem aplicadas em todas as colunas

In [50]:
functions = ['count', 'mean', 'max']

In [51]:
result = grouped['Renda', 'Anos de Estudo'].agg(functions)
result

<ipython-input-51-ba3d09886ef5>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['Renda', 'Anos de Estudo'].agg(functions)


Renda                      Anos de Estudo               
                    count         mean     max          count       mean max
Gênero    Cor2                                                              
Feminino  Amarela     117  3027.341880   20000            117  12.495726  16
          Branca     9621  2109.866750  100000           9621  11.297578  17
          Indígena    101  2464.386139  120000            101   9.108911  17
          Parda     10862  1176.758516   30000          10862   9.520714  17
          Preta      2889  1134.596400   23000           2889   9.522326  17
Masculino Amarela     235  4758.251064   50000            235  12.587234  16
          Branca    22194  2925.744435  200000          22194  10.140353  17
          Indígena    256  1081.710938   10000            256   7.875000  17
          Parda     25063  1659.577425  100000          25063   8.329929  17
          Preta      5502  1603.861687   50000           5502   8.514540  17

In [52]:
result['Renda']

count         mean     max
Gênero    Cor2                                
Feminino  Amarela     117  3027.341880   20000
          Branca     9621  2109.866750  100000
          Indígena    101  2464.386139  120000
          Parda     10862  1176.758516   30000
          Preta      2889  1134.596400   23000
Masculino Amarela     235  4758.251064   50000
          Branca    22194  2925.744435  200000
          Indígena    256  1081.710938   10000
          Parda     25063  1659.577425  100000
          Preta      5502  1603.861687   50000

- Passando uma lista de tuplas personalizando o nome das funções

In [53]:
ftuples = [('Média', 'mean'),('Desvio Padrão', 'mad')]

In [54]:
grouped['Renda', 'Anos de Estudo'].agg(ftuples)

<ipython-input-54-7dd97c9a50c6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['Renda', 'Anos de Estudo'].agg(ftuples)


Renda               Anos de Estudo              
                          Média Desvio Padrão          Média Desvio Padrão
Gênero    Cor2                                                            
Feminino  Amarela   3027.341880   2549.146322      12.495726      3.243626
          Branca    2109.866750   1670.967106      11.297578      3.427083
          Indígena  2464.386139   3007.892952       9.108911      3.971375
          Parda     1176.758516    811.580946       9.520714      3.750231
          Preta     1134.596400    705.453357       9.522326      3.658669
Masculino Amarela   4758.251064   3709.597211      12.587234      3.166971
          Branca    2925.744435   2261.012346      10.140353      3.790107
          Indígena  1081.710938    798.910889       7.875000      3.792969
          Parda     1659.577425   1125.827704       8.329929      3.851627
          Preta     1603.861687    975.602482       8.514540      3.807608

### Devolvendo dados agregados sem índices de linha

- Podemos desativar o comportamento dos dados afregados serem retornados como índices passando as_index=False para groupby

In [55]:
dados.groupby(['Gênero', 'Renda'], as_index=False).mean()

,Gênero,Renda,UF,Sexo,Idade,Cor,Anos de Estudo,Altura
0,Feminino,0,28.293153,1.0,55.505304,5.749277,5.288332,1.699083
1,Feminino,8,22.000000,1.0,49.000000,4.000000,2.000000,1.650630
2,Feminino,10,22.800000,1.0,62.000000,8.000000,4.600000,1.701249
3,Feminino,12,25.000000,1.0,55.000000,4.000000,1.000000,1.849825
4,Feminino,13,28.000000,1.0,55.000000,2.000000,1.000000,1.818524
...,...,...,...,...,...,...,...,...
2232,Masculino,60000,29.000000,0.0,56.750000,3.500000,14.500000,1.680315
2233,Masculino,80000,35.750000,0.0,57.750000,2.000000,13.000000,1.724978
2234,Masculino,90000,33.000000,0.0,58.000000,8.000000,16.000000,1.717068
2235,Masculino,100000,36.500000,0.0,62.750000,3.500000,16.000000,1.680492
